### Week 3 Part 1

### Import nesseccery libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install beautifulsoup4 --yes
from bs4 import BeautifulSoup

!conda install xml
import xml

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be UPDATED:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi              pkgs/main::certifi-2020.4.5.1-py36_0 --> conda-forge::certifi-2020.4.5.1-py36h9f0ad1d_0
  openssl              pkgs/main::openssl-1.1.1g-h7b6447c_0 --> conda-forge::openssl-1.1.1g-h516909a_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - xml

Current channels:

  - https://repo.anaconda.com/pkgs/ma

### Import data from wekipedia using BeautifulSoup

In [6]:
#get url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url).text
soup = BeautifulSoup(r)

### Find table and scrapt data

In [10]:
table = soup.find('table')
fields = table.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
    
df_post = pd.DataFrame(data=[postcode, borough, neighbourhood])
df_post

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179
0,M1A,M2A,M3A,M4A,M5A,M6A,M7A,M8A,M9A,M1B,M2B,M3B,M4B,M5B,M6B,M7B,M8B,M9B,M1C,M2C,M3C,M4C,M5C,M6C,M7C,M8C,M9C,M1E,M2E,M3E,M4E,M5E,M6E,M7E,M8E,M9E,M1G,M2G,M3G,M4G,M5G,M6G,M7G,M8G,M9G,M1H,M2H,M3H,M4H,M5H,M6H,M7H,M8H,M9H,M1J,M2J,M3J,M4J,M5J,M6J,M7J,M8J,M9J,M1K,M2K,M3K,M4K,M5K,M6K,M7K,M8K,M9K,M1L,M2L,M3L,M4L,M5L,M6L,M7L,M8L,M9L,M1M,M2M,M3M,M4M,M5M,M6M,M7M,M8M,M9M,M1N,M2N,M3N,M4N,M5N,M6N,M7N,M8N,M9N,M1P,M2P,M3P,M4P,M5P,M6P,M7P,M8P,M9P,M1R,M2R,M3R,M4R,M5R,M6R,M7R,M8R,M9R,M1S,M2S,M3S,M4S,M5S,M6S,M7S,M8S,M9S,M1T,M2T,M3T,M4T,M5T,M6T,M7T,M8T,M9T,M1V,M2V,M3V,M4V,M5V,M6V,M7V,M8V,M9V,M1W,M2W,M3W,M4W,M5W,M6W,M7W,M8W,M9W,M1X,M2X,M3X,M4X,M5X,M6X,M7X,M8X,M9X,M1Y,M2Y,M3Y,M4Y,M5Y,M6Y,M7Y,M8Y,M9Y,M1Z,M2Z,M3Z,M4Z,M5Z,M6Z,M7Z,M8Z,M9Z
1,Not assigned,Not assigned,North York,North York,Downtown Toronto,North York,Downtown Toronto,Not assigned,Etobicoke,Scarborough,Not assigned,North York,East York,Downtown Toronto,North York,Not assigned,Not assigned,Etobicoke,Scarborough,Not assigned,North York,East York,Downtown Toronto,York,Not assigned,Not assigned,Etobicoke,Scarborough,Not assigned,Not assigned,East Toronto,Downtown Toronto,York,Not assigned,Not assigned,Not assigned,Scarborough,Not assigned,Not assigned,East York,Downtown Toronto,Downtown Toronto,Not assigned,Not assigned,Not assigned,Scarborough,North York,North York,East York,Downtown Toronto,West Toronto,Not assigned,Not assigned,Not assigned,Scarborough,North York,North York,East York,Downtown Toronto,West Toronto,Not assigned,Not assigned,Not assigned,Scarborough,North York,North York,East Toronto,Downtown Toronto,West Toronto,Not assigned,Not assigned,Not assigned,Scarborough,North York,North York,East Toronto,Downtown Toronto,North York,Not assigned,Not assigned,North York,Scarborough,North York,North York,East Toronto,North York,York,Not assigned,Not assigned,North York,Scarborough,North York,North York,Central Toronto,Central Toronto,York,Not assigned,Not assigned,York,Scarborough,North York,Not assigned,Central Toronto,Central Toronto,West Toronto,Not assigned,Not assigned,Etobicoke,Scarborough,North York,Not assigned,Central Toronto,Central Toronto,West Toronto,Mississauga,Not assigned,Etobicoke,Scarborough,Not assigned,Not assigned,Central Toronto,Downtown Toronto,West Toronto,Not assigned,Not assigned,Not assigned,Scarborough,Not assigned,Not assigned,Central Toronto,Downtown Toronto,Not assigned,Not assigned,Not assigned,Not assigned,Scarborough,Not assigned,Not assigned,Central Toronto,Downtown Toronto,Not assigned,Not assigned,Etobicoke,Etobicoke,Scarborough,Not assigned,Not assigned,Downtown Toronto,Downtown Toronto,Not assigned,Not assigned,Etobicoke,Etobicoke,Scarborough,Not assigned,Not assigned,Downtown Toronto,Downtown Toronto,Not assigned,Not assigned,Etobicoke,Not assigned,Not assigned,Not assigned,Not assigned,Downtown Toronto,Not assigned,Not assigned,East Toronto,Etobicoke,Not assigned,Not assigned,Not assigned,Not assigned,Not assigned,Not assigned,Not assigned,Not assigned,Etobicoke,Not assigned
2,,,Parkwoods,Victoria Village,Regent Park / Harbourfront,Lawrence Manor / Lawrence Heights,Queen's Park / Ontario Provincial Government,,Islington Avenue,Malvern / Rouge,,Don Mills,Parkview Hill / Woodbine Gardens,Garden District / Ryerson,Glencairn,,,West Deane Park / Princess Gardens / Martin Gr...,Rouge Hill / Port Union / Highland Creek,,Don Mills,Woodbine Heights,St. Jam

In [11]:
df_post = df_post.T
df_post.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_post.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Repalce "Not assigned" with NaN in Borough column and then drop it

In [16]:
df_post['Borough'].replace('Not assigned', np.nan, inplace=True)
df_post.head()

,PostalCode,Borough,Neighborhood
0,M1A,NaN,
1,M2A,NaN,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [21]:
df_post.dropna(subset=['Borough'], axis=0, inplace=True)
df_post.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Group neighborhood

In [24]:
df_post_group = df_post.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply('.'.join).to_frame()
df_post_group.head()

,,Neighborhood
PostalCode,Borough,
M1B,Scarborough,Malvern / Rouge
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
M1E,Scarborough,Guildwood / Morningside / West Hill
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [26]:
df_post_group.reset_index(inplace=True)
df_post_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### There is no "Not assigned" in Neighborhood column

In [27]:
df_post_group[df_post_group['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


In [30]:
df_post_group.shape

(103, 3)